# **Skenario 2 - CNN ResNet50**

Skenario 2 menggunakan model Convolutional Neural Network (CNN) dengan arsitektur ResNet50 untuk melakukan klasifikasi pada data citra otak.

---

## 1. Import packages

Import modul-modul yang diperlukan untuk pengolahan citra medis, termasuk pembacaan citra dengan `SimpleITK`, manipulasi array dengan `NumPy`, visualisasi dengan `Matplotlib`, dan perhitungan entropy dengan `scipy.stats`

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import io

## 2. Data Preparation

### 2.1 Load dan Eksplorasi Data
cari dan cetak data `selected_image.npz` di dalam struktur direktori

In [3]:
directory_path = r"D:\Documents\Kuliah\.SKRIPSI\KLASIFIKASI ALZHEIMER\Coding\Notebook\Preprocessing\selected_image.npz"
loaded_data = np.load(directory_path, allow_pickle=True)

combined_slices = loaded_data['combined_slices'].item()

for label, plane_slices in combined_slices.items():
    for plane, result_slices in plane_slices.items():
        total_slices = len(result_slices)
        
        print(f"Label: {label}, Plane: {plane}, Jumlah Total Slice: {total_slices}")

        # Menampilkan bentuk (shape) dari setiap slice
        for i, (position, resized_slice) in enumerate(result_slices):
            print(f"  Resized Slice {i+1} - Position: {position}, Shape: {resized_slice.shape}")


Label: AD, Plane: coronal, Jumlah Total Slice: 60
  Resized Slice 1 - Position: 107, Shape: (224, 224)
  Resized Slice 2 - Position: 111, Shape: (224, 224)
  Resized Slice 3 - Position: 110, Shape: (224, 224)
  Resized Slice 4 - Position: 122, Shape: (224, 224)
  Resized Slice 5 - Position: 121, Shape: (224, 224)
  Resized Slice 6 - Position: 123, Shape: (224, 224)
  Resized Slice 7 - Position: 132, Shape: (224, 224)
  Resized Slice 8 - Position: 131, Shape: (224, 224)
  Resized Slice 9 - Position: 133, Shape: (224, 224)
  Resized Slice 10 - Position: 136, Shape: (224, 224)
  Resized Slice 11 - Position: 135, Shape: (224, 224)
  Resized Slice 12 - Position: 137, Shape: (224, 224)
  Resized Slice 13 - Position: 136, Shape: (224, 224)
  Resized Slice 14 - Position: 137, Shape: (224, 224)
  Resized Slice 15 - Position: 135, Shape: (224, 224)
  Resized Slice 16 - Position: 133, Shape: (224, 224)
  Resized Slice 17 - Position: 132, Shape: (224, 224)
  Resized Slice 18 - Position: 131, Shape

### 2.2 Pembagian Data
code dibawah ini digunakan untuk menyiapkan data sebelum diolah, dilakukan konversi potongan gambar hasil preprocessing dari `combined_slices` ke dalam format yang dapat digunakan untuk pelatihan model, sehingga menciptakan array `X` yang berisi potongan gambar dan array `y` yang berisi label kelas yang sesuai, dengan label kelas diubah menjadi representasi numerik menggunakan `LabelEncoder`.

In [4]:
X = []
y = []

for label, plane_slices in combined_slices.items():
    for plane, result_slices in plane_slices.items():
        X.extend([np.expand_dims(slice_data, axis=-1) for _, slice_data in result_slices])
        y.extend([label] * len(result_slices))

X = np.array(X)
y = np.array(y)

In [5]:
# Label Encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

## 3. Data Preprocessing

### 3.1 Normalisasi dan Penyesuaian Format Citra
penetapan parameter `batch_size`, lalu menggunakan `K-Fold` untuk menentukan konfigurasi dengan k-fold cross-validationnya 5 fold

In [6]:
from sklearn.model_selection import StratifiedKFold

# Perform K-Fold cross-validation
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

mean = 0.485
std = 0.229

### 3.2 Membuat Dataset dari Tensor
menggunakan TensorFlow untuk membuat dataset dari tensor (X_train_rgb, y_train) dan (X_test_rgb, y_test), kemudian mengonfigurasi dataset tersebut menjadi batch dengan ukuran 32 untuk meningkatkan efisiensi pelatihan dan penggunaan memori.

In [10]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report

# Inisialisasi k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X, y), start=1):
    # Reinitialize the model for each fold
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation='softmax'))

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # One-hot encode labels
    y_train_encoded = to_categorical(y_train, num_classes=4)
    y_test_encoded = to_categorical(y_test, num_classes=4)

    # Standardize and normalize data
    X_train_normalized = (X_train / 255.0 - mean) / std
    X_test_normalized = (X_test / 255.0 - mean) / std

    # Convert grayscale images to RGB
    X_train_rgb = np.repeat(X_train_normalized, 3, axis=-1)
    X_test_rgb = np.repeat(X_test_normalized, 3, axis=-1)

    # Train the model
    model.fit(X_train_rgb, y_train_encoded, epochs=10, validation_data=(X_test_rgb, y_test_encoded))

    # Evaluate the model on the test set
    _, accuracy_fold = model.evaluate(X_test_rgb, y_test_encoded)
    y_pred = model.predict(X_test_rgb)
    y_pred_labels = np.argmax(y_pred, axis=1)
    classification_rep_fold = classification_report(np.argmax(y_test_encoded, axis=1), y_pred_labels)

    # Display evaluation results for each fold
    print(f"Fold {i} - Accuracy: {accuracy_fold}")
    print("Classification Report:")
    print(classification_rep_fold)

    cv_scores.append(accuracy_fold)

# Display the final cross-validation results
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy: ", np.mean(cv_scores))

Epoch 1/10
18/18 [==============================] - 138s 7s/step - loss: 36.8579 - accuracy: 0.2830 - val_loss: 7.6636 - val_accuracy: 0.5833
Epoch 2/10
18/18 [==============================] - 123s 7s/step - loss: 5.5886 - accuracy: 0.3958 - val_loss: 1.1875 - val_accuracy: 0.4444
Epoch 3/10
18/18 [==============================] - 123s 7s/step - loss: 1.1882 - accuracy: 0.4427 - val_loss: 1.2432 - val_accuracy: 0.4514
Epoch 4/10
18/18 [==============================] - 126s 7s/step - loss: 1.1261 - accuracy: 0.4757 - val_loss: 0.9806 - val_accuracy: 0.5694
Epoch 5/10
18/18 [==============================] - 138s 8s/step - loss: 1.0791 - accuracy: 0.4670 - val_loss: 0.9592 - val_accuracy: 0.5833
Epoch 6/10
18/18 [==============================] - 119s 7s/step - loss: 1.0566 - accuracy: 0.4913 - val_loss: 1.0410 - val_accuracy: 0.5486
Epoch 7/10
18/18 [==============================] - 123s 7s/step - loss: 0.9910 - accuracy: 0.5035 - val_loss: 0.7955 - val_accuracy: 0.6944
Epoch 8/10
1